# Обработка отсутствующих данных

In [1]:
import numpy as np
import pandas as pd

## None: отсутствующие данные в языке Python

In [2]:
# None - это объект, поэтому общий тип массива object
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

In [3]:
for dtype in ['object', 'int']:
    print('dtype=', dtype)
    # %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype= object

dtype= int



## NaN: отсутствующие числовые данные

Еще одно представление отсутствующих данных, NaN, представляет собой специальное значение с плавающей точкой, распознаваемое всеми системами, использующими стандарное IEEE-представление чисел с плавающей точкой:

In [4]:
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

dtype('float64')

Вне зависимости от операции результат арифметического действия с участием NaN будет равен NaN:

In [5]:
1 + np.nan

nan

In [6]:
0 * np.nan

nan

In [7]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

Библиотека NumPy предоставляет специальные агрегирующие функции, игнорирующие эти пропущенные значения:

In [8]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

## Значения NaN и None в библиотеке Pandas

Как у значения NaN, так и у None есть свое назначение, и библиотека Pandas делает их практически взаимозаменяемыми путем преобразования одного в другое в определенных случаях:

In [9]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [10]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [11]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

Помимо приведения типа целочисленного массива к массиву значений с плавающей точкой, библиотека Pandas автоматически преобразует значение None в NaN.

## Операции над пустыми значениями

isnull() — генерирует булеву маску для отсутствующих значений.

notnull() — противоположность метода isnull().

dropna() — возвращает отфильтрованный вариант данных.

fillna() — возвращает копию данных, в которой пропущенные значения заполнены или восстановлены.

### Выявление пустых значений

In [13]:
data = pd.Series([1, np.nan, 'hello', None])
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [15]:
data[data.notnull()]

0        1
2    hello
dtype: object

Аналогичные булевы результаты дает использование методов isnull() и notnull() для объектов DataFrame.

### Удаление пустых значений

In [16]:
data.dropna()

0        1
2    hello
dtype: object

In [18]:
df = pd.DataFrame([[1, np.nan, 2],
                  [2, 3, 5],
                  [np.nan, 4, 6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [21]:
# по умолчанию для DataFrame dropna() отбрасывает всю строку в объекте, в которой есть объект None или NaN
df.dropna()

,0,1,2
1,2.0,3.0,5


Нельзя выбросить из DataFrame отдельные значения, только целые строки или столбцы. В зависимости от приложения может понадобиться тот или иной вариант, так что функция dropna() предоставляет для случая объектов DataFrame несколько параметров.

In [28]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


Параметры how и thresh обеспечивают точный контроль допустимого количиства пустых значений.

По умолчанию how='any', то есть отбрасываются все строки или столбцы (в зависимости от ключевого слова axis), содержащие хоть одно пустое значение. Можно также указать значение how='all', при нем будут отбрасываться только строки/столбцы, все значения в которых пустые:

In [30]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [31]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


Для более точного контроля можно задать с помощью параметра thresh минимальное количество непустых значений для строки/столбца, при котором он не отбрасывается:

In [34]:
# Отбрасываются первая и последняя строки, поскольку в них содержится только по два непустых значения.
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


### Заполнение пустых значений

In [35]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [36]:
# можно заполнить все NA-элементы одним фиксированным значением, например, нулями:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

Можно задать параметр заполнения по направлению "вперед", копирую предыдущее значение в следующую ячейку:


In [37]:
# заполнение по направлению "вперед"
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [38]:
# заполнение по направлению "назад"
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

Для объектов DataFrame опции аналогичны, но в дополнение можно задать ось, по которой будет выполняться заполнение:

In [39]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [43]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
